# BCDP Execution in U-SPS

In [1]:
import requests
sps_url = 'http://a519940d6293d447a8668ea7bf86d326-1806755561.us-west-2.elb.amazonaws.com:5001'
app_url = 'https://raw.githubusercontent.com/unity-sds/unity-analytics-bcdp/main/examples/cmip6-bucket/cwl/applicationDescriptor.json'

The endpoint for listing all deployed processes.

In [2]:
requests.get(f'{sps_url}/processes').json()

{'processes': []}

OGC Application Descriptor for BCDP Regridding

In [3]:
app_desc = requests.get(app_url).json()
app_desc

{'processDescription': {'process': {'id': 'unity-analytics-bcdp.58c0285a',
   'title': 'Readd docker req to process.cwl\n',
   'owsContext': {'offering': {'content': {'href': 'https://raw.githubusercontent.com/unity-sds/unity-analytics-bcdp/main/examples/cmip6-bucket/cwl/workflow.cwl'}}},
   'abstract': 'Application Package Demo',
   'keywords': ['Demo'],
   'inputs': [{'id': 'dataset',
     'title': 'Automatically detected using papermill.',
     'literalDataDomains': [{'dataType': {'name': 'string'}}]},
    {'id': 'backend',
     'title': 'Automatically detected using papermill.',
     'literalDataDomains': [{'dataType': {'name': 'string'}}]},
    {'id': 'method',
     'title': 'Automatically detected using papermill.',
     'literalDataDomains': [{'dataType': {'name': 'string'}}]},
    {'id': 'dlon',
     'title': 'Automatically detected using papermill.',
     'literalDataDomains': [{'dataType': {'name': 'float'}}]},
    {'id': 'dlat',
     'title': 'Automatically detected using pa

Deploy the BCDP App

In [4]:
result = requests.post(f'{sps_url}/processes', json=app_desc).json()
pid = result['deploymentResult']['processSummary']['processDescriptionURL'].split('/')[-1]
result

{'deploymentResult': {'processSummary': {'abstract': 'Application Package Demo',
   'id': 'unity-analytics-bcdp.58c0285a',
   'jobControlOptions': ['async-execute'],
   'keywords': ['Demo'],
   'processDescriptionURL': 'http://127.0.0.1:5000/processes/unity-analytics-bcdp.58c0285a:1.0.0',
   'title': 'Readd docker req to process.cwl\n',
   'version': '1.0.0'}}}

Verify success of deployment

In [5]:
requests.get(f'{sps_url}/processes').json()

{'processes': [{'abstract': 'Application Package Demo',
   'executionUnit': 1,
   'id': 'unity-analytics-bcdp.58c0285a:1.0.0',
   'immediateDeployment': 'true',
   'inputs': [{'id': 'dataset',
     'literalDataDomains': [{'dataType': {'name': 'string'}}],
     'title': 'Automatically detected using papermill.'},
    {'id': 'backend',
     'literalDataDomains': [{'dataType': {'name': 'string'}}],
     'title': 'Automatically detected using papermill.'},
    {'id': 'method',
     'literalDataDomains': [{'dataType': {'name': 'string'}}],
     'title': 'Automatically detected using papermill.'},
    {'id': 'dlon',
     'literalDataDomains': [{'dataType': {'name': 'float'}}],
     'title': 'Automatically detected using papermill.'},
    {'id': 'dlat',
     'literalDataDomains': [{'dataType': {'name': 'float'}}],
     'title': 'Automatically detected using papermill.'}],
   'jobControlOptions': ['async-execute'],
   'keywords': 'Demo',
   'outputTransmission': ['reference'],
   'outputs': [{

In [6]:
process_url = f'{sps_url}/processes/{pid}'
requests.get(process_url).json()

{'process': {'abstract': 'Application Package Demo',
  'executionUnit': 1,
  'id': 'unity-analytics-bcdp.58c0285a:1.0.0',
  'immediateDeployment': 'true',
  'inputs': [{'id': 'dataset',
    'literalDataDomains': [{'dataType': {'name': 'string'}}],
    'title': 'Automatically detected using papermill.'},
   {'id': 'backend',
    'literalDataDomains': [{'dataType': {'name': 'string'}}],
    'title': 'Automatically detected using papermill.'},
   {'id': 'method',
    'literalDataDomains': [{'dataType': {'name': 'string'}}],
    'title': 'Automatically detected using papermill.'},
   {'id': 'dlon',
    'literalDataDomains': [{'dataType': {'name': 'float'}}],
    'title': 'Automatically detected using papermill.'},
   {'id': 'dlat',
    'literalDataDomains': [{'dataType': {'name': 'float'}}],
    'title': 'Automatically detected using papermill.'}],
  'jobControlOptions': ['async-execute'],
  'keywords': 'Demo',
  'outputTransmission': ['reference'],
  'outputs': [{'id': 'nc_file',
    'out

Inputs to pass to job execution. These should match the parameters in the papermilled notebook and application descriptor, which for this example are:
- dataset: Link to S3 bucket url for dataset file
- backend: Regridding Library to use
- method: Regridding Algorithm
- dlon, dlat: lon / lat grid spacing in degrees

In [7]:
app_inputs = {
    "mode": "async",
    "response": "document",
    "inputs": [
      {
        "id": "dataset", 
        "data": "s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2-WACCM/historical/r1i1p1f1/Amon/tas/gn/v20190227/"
      }, 
      {
        "id": "backend", 
        "data": "scipy"
      }, 
      {
        "id": "method", 
        "data": "linear"
      }, 
      {
        "id": "dlon", 
        "data": 5
      }, 
      {
        "id": "dlat", 
        "data": 5
      }
    ],    
    "outputs": [
    {
      "id": "output",
      "transmissionMode": "reference"
    }
    ] 
    }

Stage the job for execution

In [8]:
# Need to do this due to bug in U-SPS with job-id
jobs_url = process_url.replace('unity-analytics-bcdp.', '') + '/jobs'
requests.post(jobs_url, json=app_inputs)

<Response [201]>

Check job status

In [9]:
requests.get(jobs_url).json()

{'jobs': [{'inputs': [{'destination': 'context',
     'name': 'dataset',
     'value': 's3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2-WACCM/historical/r1i1p1f1/Amon/tas/gn/v20190227/'},
    {'destination': 'context', 'name': 'backend', 'value': 'scipy'},
    {'destination': 'context', 'name': 'method', 'value': 'linear'},
    {'destination': 'context', 'name': 'dlon', 'value': 5},
    {'destination': 'context', 'name': 'dlat', 'value': 5}],
   'jobID': '750c3aa6-c3f8-429d-bbcf-8c8d5e1ed5ed',
   'status': 'running'},
  {'inputs': [{'destination': 'context',
     'name': 'dataset',
     'value': 's3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2-WACCM/historical/r1i1p1f1/Amon/tas/gn/v20190227/'},
    {'destination': 'context', 'name': 'backend', 'value': 'scipy'},
    {'destination': 'context', 'name': 'method', 'value': 'linear'},
    {'destination': 'context', 'name': 'dlon', 'value': 5},
    {'destination': 'context', 'name': 'dlat', 'value': 5}],
   'jobID': '8e428896-630a-4df3-9fe4-defd62b0259a',
   'sta

Remove BCDP App from deployment

In [ ]:
#requests.delete(process_url)